In [1]:
import numpy as np
from skimage import io
from skimage import exposure
from skimage import color

import supervisely_lib as sly

import os

from skimage.transform import rescale

In [2]:
# Change this field to the name of your team, where target workspace exists.
team_name = "marcwang17" # Automatically inserted

# Change this field to the of your workspace, where target project exists.
workspace_name = "First Workspace" # Automatically inserted

# If you are running this notebook on a Supervisely web instance, the connection
# Edit those values if you run this notebook on your own PC
# details below will be filled in from environment variables automatically.
#
# If you are running this notebook locally on your own machine, edit to fill in the
# connection details manually. You can find your access token at
# "Your name on the top right" -> "Account settings" -> "API token".
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [3]:
# Initialize API object
api = sly.Api(address, token)

In [4]:
project_name = 'random_sampled'

In [5]:
# Get IDs of team and workspace
team = api.team.get_info_by_name(team_name)
workspace = api.workspace.get_info_by_name(team.id, workspace_name)

In [6]:

project = api.project.get_info_by_name(workspace.id, project_name)
if project is None:
    project = api.project.create(workspace.id, project_name, change_name_if_conflict=True)



In [7]:
images_dir = './data/bimodal_batch'

In [8]:
img_paths = sly.fs.list_files(images_dir)

In [9]:
img_paths

['./data/bimodal_batch/2020-01-12 Marc 5um_5um.tiff',
 './data/bimodal_batch/2020-01-21 Marc SS Samples 5um_5um scan 1.tiff',
 './data/bimodal_batch/2020-01-21 Marc SS Samples 5um_5um scan 2_recon.tiff',
 './data/bimodal_batch/2020-01-21 Marc SS Samples 5um_5um scan 3_recon.tiff',
 './data/bimodal_batch/2020-01-21 Marc SS Samples 5um_5um scan 4.tiff']

In [10]:
im_size = 100
img_sample = 3

In [11]:
def axis_slice(im, i, j, k, size, axis):
    if axis == 0:
        return im[i,j:j+size, k:k+size]
    elif axis == 1:
        return im[i:i+size, j, k:k+size]
    elif axis == 2:
        return im[i:i+size, j:j+size, k]
    else:
        return None


In [12]:
np.random.seed(0)

for i, img_path in enumerate(img_paths):
    full_im = io.imread(img_path)
    
    full_im = exposure.rescale_intensity(full_im, in_range=np.uint16, out_range=np.uint8)
    
    file_name = sly.fs.get_file_name(img_path)
    dataset = api.dataset.create(project.id, file_name, change_name_if_conflict=True)
    ls_name = []
    ls_im = []
    
    shape = full_im.shape
    for j in range(img_sample):
        axis = np.random.randint(0,3)
        
        if(axis == 0):
            z_max = int(shape[0])
            x_max = int(shape[1]-im_size)
            y_max = int(shape[2]-im_size)
        elif(axis == 1):
            z_max = int(shape[0]-im_size)
            x_max = int(shape[1])
            y_max = int(shape[2]-im_size)
        elif(axis == 2):
            z_max = int(shape[0]-im_size)
            x_max = int(shape[1]-im_size)
            y_max = int(shape[2])
        
        z_init = np.random.randint(im_size, z_max)
        x_init = np.random.randint(im_size, x_max)
        y_init = np.random.randint(im_size, y_max)
    
        sampled_im = axis_slice(full_im, z_init, x_init, y_init, im_size, axis)
        
        upsampled = rescale(sampled_im, scale = 6, preserve_range = True)
        
        rgb_im = color.gray2rgb(upsampled)
        ls_name.append('{}_{}.png'.format(file_name, j))
        ls_im.append(rgb_im)
        
    api.image.upload_nps(dataset.id, ls_name, ls_im)
        